In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [51]:
from py2neo import Graph, Node, Relationship, RelationshipMatcher,RelationshipMatch
password="dfg"
ip='163.172.110.238'
graph = Graph('bolt://'+ip, auth=("neo4j", password),name="validalabdev")

In [4]:
def switch_entity_name(entity_type):
    """[summary]
    Args:
        entity_type (str): [description]
    Returns:
        str: [description]
    """
    switcher={
                "entity":'entity_name',
                "website":'site_name',
                "youtube":'user_name',
                "socialmedia":'user_name',
                "wikipedia":'',
             }
    return switcher.get(entity_type.lower(),"Not a social media")

### A regarder

In [153]:
def find_node(entity_type,merge_key,property_name):
    return graph.nodes.match(entity_type).where("_."+merge_key+" =~ '.*"+property_name+".*'").first()

def create_and_add_node(graph,entity_type,merge_key,property_name,node=None):
    from graphio import NodeSet, RelationshipSet
    if node==None:       
        node = NodeSet([entity_type], merge_keys=[merge_key])
        node.add_node({merge_key:property_name})
        
        print(node,property_name )
    else:
        node.add_node({merge_key:property_name})
        print(node,property_name, 'ELse')
    node.merge(graph)
    return node

def add_and_push_node(graph,merge_key,property_name,entity_type,node=None): 
    if find_node(entity_type,merge_key,property_name):
        pass
    else:
        create_and_add_node(graph,entity_type,merge_key,property_name,node)

### A regarder

In [154]:
def get_entity_name(link):
    from urllib.parse import urlparse 
    """
    """
    if 'facebook' in link.lower() or 'fb.' in link.lower() :
        entity_name=link.split('/')[3].split('?')[0].lower()
        entity_address=link.split('?')[0].lower().replace("www.",'').lower()
        entity_type="Facebook"
    elif 'twitter' in link.lower():
        entity_name=link.split('/')[3].split('?')[0].lower()
        entity_address=link.split('?')[0].lower()
        entity_type='Twitter'
    else:
        if len(link.split('/'))<2:
            entity_name=link.lower().replace("www.",'')
            parsed_uri = urlparse(link)  # returns six components
            entity_address = 'https:'+'//'+link.lower()
            
        else:
            entity_name=link.split('/')[2].split('?')[0].lower().replace("www.",'')
            parsed_uri = urlparse(link)  # returns six components
            entity_address = link.split('/')[0]+'//'+link.split('/')[2]
#         '{uri.netloc}/'.format(uri=parsed_uri).split('/')[0].lower()
        entity_type='Website'
    return {'name':entity_name,'address':entity_address,'type':entity_type}

### A regarder

In [157]:
def push_relations(nameOfRelation,sourceType,targetType,sourceProperty,targetProperty,sourceMetadata,targetMetadata):
    from graphio import NodeSet, RelationshipSet
    sourceData=get_entity_name(sourceMetadata['source'])
    
    sourceNode = NodeSet([sourceType], merge_keys=[sourceProperty])
    targetNode = NodeSet([targetType], merge_keys=[targetProperty])
    
    add_and_push_node(graph,sourceProperty,sourceData['name'],sourceType,sourceNode)

    RELATIONSHIP = RelationshipSet(nameOfRelation,
                                                   [sourceType], [targetType],
                                                   [sourceProperty], [targetProperty]) 
    print("""RelationshipSet({},       [{}], [{}],
                                                   [{}], [{}]) """.format(nameOfRelation,sourceType,targetType,sourceProperty,targetProperty))
    
    for item in targetMetadata:

        targetNode=add_and_push_node(graph,targetProperty,item['name'],targetType,targetNode)

        RELATIONSHIP.add_relationship({sourceProperty:sourceData['name']},
                                                         {targetProperty:item['name']},
                                                         sourceMetadata) 
    RELATIONSHIP.merge(graph)


In [158]:
url_list=['test2.com','www.atlantic-test2.com','www.test2-finance.fr','www.eco-test2.cash']
# url_list=['soromabnkfinance.group','www.atlantic-credit.com','www.courtin-finance.fr','www.eco-credit.cash']
targetMetadata=[get_entity_name(entity) for entity in url_list]
metadata={'meaning': 'testParo', 'weigth':'-10', 'source':'https://www.abe-infoservice.fr/liste-noire/listes-noires-et-alertes-des-autorites'}
# push_recommands('RECOMMANDS',start_node='Website',end_node='Website',property_start='name',property_end='name',source='abe-infoservice.fr',url_list,metadata)
push_relations(nameOfRelation='RECOMMENDS',sourceType='Website',targetType='Website',sourceProperty='name',targetProperty='name',sourceMetadata=metadata,targetMetadata=targetMetadata)

RelationshipSet(RECOMMENDS,       [Website], [Website],
                                                   [name], [name]) 


In [150]:
t=find_node('Website','name','abe-infoservice.fr')
x=find_node('Website','name','test.com')

In [152]:
relMatch = RelationshipMatcher(graph)
list(relMatch.match((t,None),'RECOMMENDS'))

[RECOMMENDS(Node('Website', ArticleRank=0.1591661642633618, BetweennessCentrality=0.0, ClosenessCentrality=0.0, D1_crawled=[False], D1_crawling_status=['UNCRAWLED'], D1_creation_date=[1589839344576.0], D1_id=['4360'], D1_indegree=[2], D1_indexing_status=['UNINDEXED'], D1_label=['Abe-Infoservice.fr'], D1_last_modification_date=[1589839344576.0], D1_name=['Abe-Infoservice.fr'], D1_outdegree=[0], D1_pages_crawled=[0], D1_pages_total=[2], D1_status=['DISCOVERED'], D1_undirected_degree=[2], DegreeCentrality=0.0, EigenvectorCentrality=0.8001903111373021, LabelPropagationCommunity=8358, LouvainCommunity=6012, ModularityOptimization=6012, PageRank=0.15979922041574848, PageRanktot=0.15846837437909675, StronglyConnectedComponents=6012, TriangleCount=1, WCC_bij=6012, WeaklyConnectedComponents=0, name='abe-infoservice.fr', site_name='abe-infoservice.fr'), Node('Website', name='atlantic-test2.com'), meaning='testParo', source='https://www.abe-infoservice.fr/liste-noire/listes-noires-et-alertes-des-

In [148]:
def push_relations(nameOfRelation,sourceType,targetType,sourceProperty,targetProperty,sourceMetadata,targetMetadata):
    from graphio import NodeSet, RelationshipSet
    sourceData=get_entity_name(sourceMetadata['source'])
#     print(sourceData)
#     sourceNode = NodeSet([sourceType], merge_keys=[sourceProperty])
    targetNode = NodeSet([targetType], merge_keys=[targetProperty])
#     print(sourceNode)
    #ajouter la source si elle n'existe pas
    sourceNode=is_node_existing(graph,sourceProperty,sourceData['name'],sourceType)
#     targetNode=is_node_existing(targetProperty,,targetType)
#     sourceNode.add_node({sourceProperty:sourceData['name']})
#     sourceNode.merge(graph)
#     add_node(propertyStart,source['name'],source['type'])
    # itérer les target et ajouter les relations
    
    
    RELATIONSHIP = RelationshipSet(nameOfRelation,
                                                   [sourceType], [targetType],
                                                   [sourceProperty], [targetProperty]) 
    print("""RelationshipSet({},       [{}], [{}],
                                                   [{}], [{}]) """.format(nameOfRelation,sourceType,targetType,sourceProperty,targetProperty))
    
    for item in targetMetadata:
#         targetNode = NodeSet([endNode], merge_keys=[propertyEnd])
#         print(item)
        targetNode=is_node_existing(graph,targetProperty,item['name'],targetType,targetNode)
#         targetNode.add_node({targetProperty:item['name']})
        RELATIONSHIP.add_relationship({sourceProperty:sourceData['name']},
                                                         {targetProperty:item['name']},
                                                         sourceMetadata) 
        
#     RELATIONSHIP.create_index(graph)
    RELATIONSHIP.merge(graph)
    print(targetNode)
#     targetNode.merge(graph)
# url_list=['soromabnkfinance.group','www.atlantic-credit.com','www.courtin-finance.fr','www.eco-credit.cash']
# # url_list=['soromabnkfinance.group','www.atlantic-credit.com','www.courtin-finance.fr','www.eco-credit.cash']
# targetMetadata=[get_entity_name(entity) for entity in url_list]
# metadata={'meaning': 'parodique', 'weigth':'-1', 'source':'https://www.abe-infoservice.fr/liste-noire/listes-noires-et-alertes-des-autorites'}
# # push_recommands('RECOMMANDS',start_node='Website',end_node='Website',property_start='name',property_end='name',source='abe-infoservice.fr',url_list,metadata)
# push_relations(nameOfRelation='RECOMMENDS',sourceType='Website',targetType='Website',sourceProperty='name',targetProperty='name',sourceMetadata=metadata,targetMetadata=targetMetadata)